In [1]:
import pandas as pd
from sqlalchemy.sql import text

from py.db import engine

In [2]:
with engine.connect() as conn:
    # statement = text("""ROLLBACK;""")

    query = "SELECT * FROM melding;"
    df = pd.read_sql(query, conn)
    
    df.to_pickle("./data/flitsers.pkl")